# Convolutional Neural Network
**get datasets from udemy**

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [2]:
tf.__version__

'2.3.1'

## Data Preprocessing

### Training Dataset
- **target_size** and batch_size of training and testing should be same

- **target_size**(images from cat folder, images from dog folder)

- **batch size** defines the number of samples that will be propagated through the network.

- For instance, let's say you have 1050 training samples and you want to set up a batch_size equal to 100. The algorithm takes the first 100 samples (from 1st to 100th) from the training dataset and trains the network. Next, it takes the second 100 samples (from 101st to 200th) and trains the network again. We can keep doing this procedure until we have propagated all samples through of the network

- 32 is the default value of batch_size

- **class_mode = 'binary**  
-- when we have only 2 classes to predict (let say cat and dog)

- **class_mode = 'classification'**  
-- when we have more than 2 classes to predict (let say cat,dog and elephant)

- **1./255 divide all pixels with 255 (converts all pixels values between 0 and 1)**

- **shear_range , zoom_range and horizontal_flip** used for image augmentation (used to avid overfitting)

- Image Augmentation is a technique that can be used to artificially expand the size of a training dataset by creating modified versions of images in the dataset

- Dont apply image augmentation on test set as testing should be on original images

In [5]:
training_data_gen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
training_data = training_data_gen.flow_from_directory(
    '../../Datasets/cnn/cnn_training_data', target_size=(64, 64), batch_size=32, class_mode='binary'
)

Found 8000 images belonging to 2 classes.


### Test Dataset

In [6]:
test_data_gen = ImageDataGenerator(rescale=1./255)
test_data = training_data_gen.flow_from_directory(
    '../../Datasets/cnn/cnn_test_data', target_size=(64, 64), batch_size=32, class_mode='binary'
)

Found 2000 images belonging to 2 classes.


## Building the CNN
- **reshape** used for feature scaling of images

- feature scaling is compulsory for training neural networks

- **kernel_size** used to define matrix size for feature detection filters (32 is recommended) 
- **filters** used to define filters in the features of images 
- **activation** used to define activation function 
- **input_shape[same size of training and test target size, 3 for color images/1 for black and white images]** used to define shape of input images 
- **pool_size** used to define max pooling matrix size in feature map 
- **strides** used to define next max pooling matrix within feature map 
- **input_shape should only be defined in first convolution layer**

### Initializing the CNN

In [12]:
cnn = tf.keras.models.Sequential()

### Convolution

In [13]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Pooling
- pooling is mainly used to prevent overfitting by removing extra features from the image

- pooling reduces the image size

- pooling is used to get invariants(tilts, twists etc) of the image instead of ideal scenario. but pooling prserved all main featured of the image which is called max pooling.

In [14]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding second convolutional layer

In [16]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Flattening

In [17]:
cnn.add(tf.keras.layers.Flatten())

### Full Connection

In [18]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Adding the output layer


In [19]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the CNN

### Compiling the CNN

In [20]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the training data and evaluating it on test data

In [21]:
cnn.fit(x=training_data, validation_data=test_data, epochs=25)

Epoch 1/25
250/250 [==============================] - 42s 167ms/step - loss: 0.6775 - accuracy: 0.5770 - val_loss: 0.7067 - val_accuracy: 0.5500
Epoch 2/25
250/250 [==============================] - 41s 163ms/step - loss: 0.6147 - accuracy: 0.6634 - val_loss: 0.5783 - val_accuracy: 0.7020
Epoch 3/25
250/250 [==============================] - 40s 162ms/step - loss: 0.5638 - accuracy: 0.7054 - val_loss: 0.5483 - val_accuracy: 0.7305
Epoch 4/25
250/250 [==============================] - 41s 162ms/step - loss: 0.5358 - accuracy: 0.7266 - val_loss: 0.6160 - val_accuracy: 0.6505
Epoch 5/25
250/250 [==============================] - 40s 161ms/step - loss: 0.5160 - accuracy: 0.7436 - val_loss: 0.5915 - val_accuracy: 0.6920
Epoch 6/25
250/250 [==============================] - 41s 162ms/step - loss: 0.4906 - accuracy: 0.7604 - val_loss: 0.5159 - val_accuracy: 0.7375
Epoch 7/25
250/250 [==============================] - 41s 162ms/step - loss: 0.4817 - accuracy: 0.7634 - val_loss: 0.4954 - val_ac

## Prediction on user data
- **expand_dims** used to handle batches target_size of prediction should be same as training and testing target_size used to define size of input image (previous definition is wrong)

In [37]:
test_image = image.load_img('../../Datasets/cnn/cnn_user_test_image_dog.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
prediction = cnn.predict(test_image)
training_data.class_indices  # used to get indices for prediction classed i.e 0=cat, 1=dog used by model
if prediction[0][0] == 1:
    print('Dog')
else:
    print('Cat')

Dog
